# Evaluation notebook for designed binders

### Simulate PDB

In [21]:
import openmm as mm
import openmm.app as app
import openmm.unit as unit
from pdbfixer import PDBFixer
from openmm.app import PDBFile

# Set PDB name and file paths
pdb_name = '6vja_chainD_chainI'

input_filename = f'../data/structures/{pdb_name}.pdb'
fixed_filename = f'../data/structures/{pdb_name}_fixed.pdb'
minimized_filename = f'../data/structures/{pdb_name}_minimized.pdb'
log_filename = f'../data/simulations/logs/{pdb_name}.log'
trajectory_filename = f'../data/simulations/trajectories/{pdb_name}.dcd'

# Set the simulation parameters
pH = 7.0
temperature = 300*unit.kelvin
friction = 1.0/unit.picoseconds
timestep = 0.002*unit.picoseconds
logsteps = 10000
trajsteps = 10000
totalsteps = 100000

# Load and fix the PDB file using PDBFixer
fixer = PDBFixer(filename=input_filename)

# Identify and fix missing residues/atoms
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()

# Add missing hydrogens
fixer.addMissingHydrogens(pH=pH)

# Remove heterogens, such as ligands (if needed)
fixer.removeHeterogens(keepWater=False)

# Save the fixed PDB file
with open(fixed_filename, 'w') as outfile:
    PDBFile.writeFile(fixer.topology, fixer.positions, outfile)

# Load the fixed structure into OpenMM
pdb = app.PDBFile(fixed_filename)
forcefield = app.ForceField('amber99sbildn.xml', 'amber99_obc.xml')

# Create the system with implicit solvent
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.CutoffNonPeriodic, 
                                 nonbondedCutoff=1.0*unit.nanometers, constraints=app.HBonds)

# Set up the Langevin integrator
integrator = mm.LangevinIntegrator(temperature, friction, timestep)

# Create the simulation object
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

# Minimize energy
print('Minimizing...')
simulation.minimizeEnergy()

# Set up reporters to log the simulation state data to a file
with open(log_filename, 'w') as logfile:
    simulation.reporters.append(app.StateDataReporter(logfile, logsteps, step=True, potentialEnergy=True, 
                                                      temperature=True, progress=True, remainingTime=True, 
                                                      totalSteps=totalsteps, separator='\t'))
    
    # Set up the DCD file reporter for saving the trajectory
    simulation.reporters.append(app.DCDReporter(trajectory_filename, trajsteps))

    # Run the simulation
    print('Running Simulation...')
    simulation.step(totalsteps)

# Save the minimized structure
positions = simulation.context.getState(getPositions=True).getPositions()
with open(minimized_filename, 'w') as outfile:
    PDBFile.writeFile(simulation.topology, positions, outfile)

print(f'Simulation complete. Log saved to {log_filename} and trajectory saved to {trajectory_filename}')

Minimizing...
Running Simulation...
Simulation complete. Log saved to ../data/simulations/logs/6vja_chainD_chainI.log and trajectory saved to ../data/simulations/trajectories/6vja_chainD_chainI.dcd


In [22]:
import openmm as mm
import openmm.app as app
import openmm.unit as unit
from pdbfixer import PDBFixer
from openmm.app import PDBFile

# Set PDB name and file paths
pdb_name = 'fold_cd20_design1_6vja_model_0'

input_filename = f'../results/fold_cd20_design1_6vja/{pdb_name}.cif'
fixed_filename = f'../results/fold_cd20_design1_6vja/{pdb_name}_fixed.pdb'
minimized_filename = f'../results/fold_cd20_design1_6vja/{pdb_name}_minimized.pdb'
log_filename = f'../data/simulations/logs/{pdb_name}.log'
trajectory_filename = f'../data/simulations/trajectories/{pdb_name}.dcd'

# Set the simulation parameters
pH = 7.0
temperature = 300*unit.kelvin
friction = 1.0/unit.picoseconds
timestep = 0.002*unit.picoseconds
logsteps = 10000
trajsteps = 10000
totalsteps = 100000

# Load and fix the PDB file using PDBFixer
fixer = PDBFixer(filename=input_filename)

# Identify and fix missing residues/atoms
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()

# Add missing hydrogens
fixer.addMissingHydrogens(pH=pH)

# Remove heterogens, such as ligands (if needed)
fixer.removeHeterogens(keepWater=False)

# Save the fixed PDB file
with open(fixed_filename, 'w') as outfile:
    PDBFile.writeFile(fixer.topology, fixer.positions, outfile)

# Load the fixed structure into OpenMM
pdb = app.PDBFile(fixed_filename)
forcefield = app.ForceField('amber99sbildn.xml', 'amber99_obc.xml')

# Create the system with implicit solvent
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.CutoffNonPeriodic, 
                                 nonbondedCutoff=1.0*unit.nanometers, constraints=app.HBonds)

# Set up the Langevin integrator
integrator = mm.LangevinIntegrator(temperature, friction, timestep)

# Create the simulation object
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

# Minimize energy
print('Minimizing...')
simulation.minimizeEnergy()

# Set up reporters to log the simulation state data to a file
with open(log_filename, 'w') as logfile:
    simulation.reporters.append(app.StateDataReporter(logfile, logsteps, step=True, potentialEnergy=True, 
                                                      temperature=True, progress=True, remainingTime=True, 
                                                      totalSteps=totalsteps, separator='\t'))
    
    # Set up the DCD file reporter for saving the trajectory
    simulation.reporters.append(app.DCDReporter(trajectory_filename, trajsteps))

    # Run the simulation
    print('Running Simulation...')
    simulation.step(totalsteps)

# Save the minimized structure
positions = simulation.context.getState(getPositions=True).getPositions()
with open(minimized_filename, 'w') as outfile:
    PDBFile.writeFile(simulation.topology, positions, outfile)

print(f'Simulation complete. Log saved to {log_filename} and trajectory saved to {trajectory_filename}')

Minimizing...
Running Simulation...
Simulation complete. Log saved to ../data/simulations/logs/fold_cd20_design1_6vja_model_0.log and trajectory saved to ../data/simulations/trajectories/fold_cd20_design1_6vja_model_0.dcd
